In [178]:
import pandas as pd
# the original intersection data that will be enriched with addtional variables
inter_collis_df = pd.DataFrame.from_csv("./OBJ2B_collisions_intersections.csv")

# alex's predicted street exposure (adt) that will be joined with the streets column of intersection data
street_exp_pred_df = pd.DataFrame.from_csv("../o1_se_result2.tsv", sep="\t")

# original street exposure data that will be selectively joined with streets column of intersection data
street_exp_orig_df = pd.DataFrame.from_csv("./OBJ1_streets_exposure.csv")

# index street data on unitidsort
street_exp_orig_df = street_exp_orig_df.set_index(['unitidsort'])

In [179]:
# join in all vars from predicted exposure
# results in a new column with list of lists: outer list for each street and inner lists of predicted exposure vars
inter_collis_df["exp_pred_vars"] = inter_collis_df.streets.apply(lambda x: [street_exp_pred_df.ix[int(y)] for y in x.split(',')])

In [180]:
inter_collis_df["exp_pred_vars"].head(2)

0    [[13345.0, 13312.4799607, 9.49645721715], [162...
1    [[nan, 3076.16862581, 8.03144014896], [nan, 39...
Name: exp_pred_vars, dtype: object

In [181]:
# join in all vars from predicted exposure
# results in a new column with list of lists: outer list for each street and inner lists of predicted exposure vars
inter_collis_df["street_vars"] = inter_collis_df.streets.apply(lambda x: [street_exp_orig_df.ix[int(y)] for y in x.split(',')])

In [182]:
inter_collis_df["street_vars"].head(2)

0    [[10075, DEXTER AVE N, DEXTER AVE N BETWEEN RE...
1    [[9725, COLUMBIA ST, COLUMBIA ST BETWEEN ALASK...
Name: street_vars, dtype: object

In [183]:
# next we want to perform some aggregations

# these are helper functions that will be used to aggregate over a particular column of street-level data
import math
import sys

def sum_over_streets(column, street_data):
    result = 0
    for street in street_data:
        result += street[column]
        
    return result

def uniq_values(column, street_data):
    values = set()
    for street in street_data:
        values.add(street[column])
        
    return len(values)

def min_known_value(column, street_data):
    minimum = sys.maxint
    for street in street_data:
        if not math.isnan(street[column]):
            if street[column] < minimum:
                minimum = street[column]
        
    if minimum == sys.maxint:
        return float('NaN')
    else:
        return minimum

def min_nonzero_value(column, street_data):
    minimum = sys.maxint
    for street in street_data:
        if street[column] != 0 and street[column] < minimum:
            minimum = street[column]
        
    if minimum == sys.maxint:
        return float('NaN')
    else:
        return minimum

def avg_of_known_values(column, street_data):
    total = 0
    count = 0
    for street in street_data:
        if not math.isnan(street[column]):
            total += street[column]
            count += 1
    if count == 0:
        return float('NaN')
    else:
        return total / float(count)
    
def avg_of_nonzero_values(column, street_data):
    total = 0
    count = 0
    for street in street_data:
        if not street[column] == 0:
            total += street[column]
            count += 1
    if count == 0:
        return float('NaN')
    else:
        return total / float(count)

def bool_true_count(column, street_data):
    count = 0
    for street in street_data:
        if street[column]:
            count += 1
        
    return count

In [184]:
# add variable intersection_adt_hat2: adt_hat2 summed over all streets (and divided by 2)
inter_collis_df["intersection_adt_hat2"] = inter_collis_df.exp_pred_vars.apply(lambda x: sum_over_streets(1, x) / 2.0)

In [185]:
# add variable zoning_cats: number of unique zoning categories over all streets divided by number of streets
inter_collis_df["zoning_cats"] = inter_collis_df.street_vars.apply(lambda x: uniq_values(4, x) / float(len(x)))

In [186]:
# add variable min_pavement_cond: min known value of pavement condition index over all streets (some are unknown)
inter_collis_df["min_pavement_cond"] = inter_collis_df.street_vars.apply(lambda x: min_nonzero_value(5, x))

In [187]:
# add variable avg_pavement_cond: avg value of known pavement condition index over all streets (some are unknown)
inter_collis_df["avg_pavement_cond"] = inter_collis_df.street_vars.apply(lambda x: avg_of_nonzero_values(5, x))

In [188]:
# add variable pct_truck_routes: percentage of truck routes over all streets
inter_collis_df["pct_truck_routes"] = inter_collis_df.street_vars.apply(lambda x: bool_true_count(6, x) / float(len(x)))

In [189]:
# add variable min_surface_width: min value of known surface widths over all streets (some are unknown)
inter_collis_df["min_surface_width"] = inter_collis_df.street_vars.apply(lambda x: min_known_value(9, x))

In [190]:
# add variable avg_surface_width: average value of known surface widths over all streets (some are unknown)
inter_collis_df["avg_surface_width"] = inter_collis_df.street_vars.apply(lambda x: avg_of_known_values(9, x))

In [191]:
# add variable pct_oneway: percentage of oneway over all streets
inter_collis_df["pct_oneway"] = inter_collis_df.street_vars.apply(lambda x: bool_true_count(10, x) / float(len(x)))

In [192]:
inter_collis_df["intersection_adt_hat2"].head(2)

0    35221.303413
1     8186.717943
Name: intersection_adt_hat2, dtype: float64

In [193]:
inter_collis_df["pct_oneway"].head(10)

0    0.250000
1    0.500000
2    0.000000
3    0.000000
4    0.500000
5    0.000000
6    0.000000
7    0.500000
8    0.000000
9    0.333333
Name: pct_oneway, dtype: float64

In [194]:
inter_collis_df

,gid,segkey,intkey,location,incdttm,yr,mon,hr,dow,col_type,...,exp_pred_vars,street_vars,intersection_adt_hat2,zoning_cats,min_pavement_cond,avg_pavement_cond,pct_truck_routes,min_surface_width,avg_surface_width,pct_oneway
0,38565,10076.0,28767,DEXTER AVE N AND MERCER ST,5/26/2006 2:35:00 PM,2006.0,5.0,14.0,5.0,Cycles,...,"[[13345.0, 13312.4799607, 9.49645721715], [162...","[[10075, DEXTER AVE N, DEXTER AVE N BETWEEN RE...",35221.303413,0.500000,26.0,70.333333,0.250000,28.0,51.000000,0.250000
1,48114,9726.0,30352,COLUMBIA ST AND WESTERN AVE,1/25/2005 5:10:00 PM,2005.0,1.0,17.0,2.0,Cycles,...,"[[nan, 3076.16862581, 8.03144014896], [nan, 39...","[[9725, COLUMBIA ST, COLUMBIA ST BETWEEN ALASK...",8186.717943,0.500000,18.0,36.250000,0.000000,34.0,39.500000,0.500000
2,12,3659.0,33983,24TH AVE SW AND SW HOLDEN ST,9/4/2008,2008.0,9.0,NaN,4.0,Cycles,...,"[[1144.0, 1186.61833439, 7.0788628049], [1370....","[[3658, 24TH AVE SW, 24TH AVE SW BETWEEN SW WE...",7133.591660,0.250000,6.0,42.000000,0.000000,23.0,33.250000,0.000000
3,23,8973.0,37326,AURORA AVE N AND N 94TH ST,1/16/2008 12:02:00 PM,2008.0,1.0,12.0,3.0,Cycles,...,"[[nan, 35456.1592898, 10.4760522627], [40130.0...","[[8972, AURORA AVE N, AURORA AVE N BETWEEN N 9...",37807.610313,0.333333,90.0,90.000000,0.666667,30.0,30.000000,0.000000
4,28,1411.0,25790,11TH AVE NE AND NE 50TH ST,6/9/2008,2008.0,6.0,NaN,1.0,Pedestrian,...,"[[7103.0, 7225.37454417, 8.88535435166], [nan,...","[[1410, 11TH AVE NE, 11TH AVE NE BETWEEN NE 47...",26798.066920,0.500000,60.0,77.750000,0.000000,40.0,40.000000,0.500000
5,36,8985.0,36360,AURORA AVE N AND N NORTHGATE WAY,6/23/2008 5:14:00 PM,2008.0,6.0,17.0,1.0,Pedestrian,...,"[[35556.0, 36104.1848969, 10.4941640627], [348...","[[8984, AURORA AVE N, AURORA AVE N BETWEEN N 1...",59413.562338,0.250000,100.0,100.000000,1.000000,40.0,48.000000,0.000000
6,74,9312.0,29940,BOREN AVE AND COLUMBIA ST,12/10/2008 1:06:00 PM,2008.0,12.0,13.0,3.0,Pedestrian,...,"[[nan, 18565.939044, 9.82908394675], [19328.0,...","[[9311, BOREN AVE, BOREN AVE BETWEEN CHERRY ST...",20600.394697,0.250000,12.0,51.000000,0.000000,30.0,38.000000,0.000000
7,122,11068.0,28661,JOHN ST AND QUEEN ANNE AVE N,7/24/2008 1:36:00 PM,2008.0,7.0,13.0,4.0,Cycles,...,"[[1544.0, 1563.7808079, 7.35486176288], [9327....","[[11068, JOHN ST, JOHN ST BETWEEN QUEEN ANNE A...",11349.514696,0.250000,18.0,34.750000,0.000000,32.0,37.000000,0.500000
8,139,4375.0,30480,3RD AVE AND JAMES ST,9/29/2008 3:54:00 PM,2008.0,9.0,15.0,1.0,Pedestrian,...,"[[6932.0, 6991.98010705, 8.85251907223], [7583...","[[4373, 3RD AVE, 3RD AVE BETWEEN YESLER WAY AN...",15337.621962,0.250000,60.0,75.500000,0.000000,42.0,44.000000,0.000000
9,171,2031.0,27145,15TH AVE NE AND NE CAMPUS WB PY,9/19/2008 3:05:00 PM,2008.0,9.0,15.0,5.0,Pedestrian,...,"[[nan, 18309.5685432, 9.8151790734], [13497.0,...","[[2030, 15TH AVE NE, 15TH AVE NE BETWEEN NE CA...",17282.761312,0.666667,49.0,82.333333,0.000000,34.0,52.666667,0.333333


In [195]:
# first drop temporary columns
inter_collis_df_final = inter_collis_df.drop('street_vars', axis=1)
inter_collis_df_final.drop('exp_pred_vars', axis=1, inplace=True)

# write to file
inter_collis_df_final.to_csv('./collisions_intersections_rod.tsv', sep="\t", index=False)